## 4.15 Vai pra lá ou vem pra cá!


Alunos: José Victor da Silva izidório

Objetivo: Implemente o operador genético de migração no código de algoritmo
genético desenvolvido nesta disciplina (isto é, não é para usar o DEAP). Conte para o
leitor sobre como a sua implementação funciona e mostre ela em ação.

Para resolver esse problema, vou aproveitar a Fera 4.12 que encontra palíndromos de 5 letras que possuam pelo menos uma vogal. Sendo assim, irei importar as mesmas funções. A idéia é utilizar a mesma estrutura, mas, dessa vez, irei utilizar um operador de migração. O objetivo aqui não será comparar a performance, já que isso requer um algoritmo mais otimizado para realizar diversas amostragens, então irei focar no operador em si.

## Operador de Migração em Algoritmos Genéticos

### O que é?

O operador de migração é uma técnica usada em algoritmos genéticos que trabalham com várias populações paralelas, chamadas de ilhas. Cada ilha evolui de forma independente, mas periodicamente alguns indivíduos são trocados entre elas.

### Para que serve?

- **Manter a diversidade genética** entre populações.
- **Evitar convergência prematura** em soluções locais.
- **Compartilhar boas soluções** entre as populações para acelerar a busca pelo ótimo.

### Como foi implementado neste código?

No código:

- Temos 10 ilhas (populações) evoluindo paralelamente.
- A cada intervalo definido (`MIGRACAO_INTERVALO`), ocorre a migração.
- Os melhores indivíduos de cada ilha são selecionados para migrar.
- Esses indivíduos substituem os piores indivíduos da ilha vizinha (seguindo um ciclo entre as ilhas).
- Isso promove a renovação genética e a troca de boas características entre as populações.




In [8]:
import random
from string import ascii_lowercase
from pprint import pprint

from funcoes_fera_4_12 import populacao_palindromo as cria_populacao
from funcoes_fera_4_12 import funcao_objetivo_pop_palindromo as funcao_objetivo
from funcoes_fera_4_12 import selecao_torneio_min as funcao_selecao
from funcoes_fera_4_12 import cruzamento_uniforme_com_vogal as funcao_cruzamento
from funcoes_fera_4_12 import mutacao_simples as funcao_mutacao1
from funcoes_fera_4_12 import mutacao_salto as funcao_mutacao2
from funcoes_fera_4_12 import migracao

In [9]:
CARACTERES_POSSIVEIS = ascii_lowercase

TAMANHO_POPULACAO = 100
TAMANHO_PALINDROMO = 5
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.025
TAMANHO_TORNEIO = 3
MIGRACAO_INTERVALO = 20
NUM_MIGRANTES = 5


NUM_ILHAS = 10
populacoes = [cria_populacao(TAMANHO_POPULACAO, TAMANHO_PALINDROMO, CARACTERES_POSSIVEIS) for _ in range(NUM_ILHAS)]
menores_fitness_gerais = [float("inf")] * NUM_ILHAS
geracoes = [0] * NUM_ILHAS

MIGRACAO_INTERVALO = 20 



In [10]:
len(populacoes)

10

In [11]:
while all(fitness > 0 for fitness in menores_fitness_gerais):
    for i in range(NUM_ILHAS):
        if menores_fitness_gerais[i] == 0:
            continue

        fitness = funcao_objetivo(populacoes[i])
        selecionados = funcao_selecao(populacoes[i], fitness, TAMANHO_TORNEIO)

        proxima_geracao = []
        for pai, mae in zip(selecionados[::2], selecionados[1::2]):
            individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
            proxima_geracao.append(individuo1)
            proxima_geracao.append(individuo2)

        funcao_mutacao1(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))
        funcao_mutacao2(proxima_geracao, CHANCE_DE_MUTACAO, list(CARACTERES_POSSIVEIS))

        populacoes[i] = proxima_geracao
        geracoes[i] += 1

        fitness = funcao_objetivo(populacoes[i])
        menor_fitness_observado = min(fitness)

        if menor_fitness_observado < menores_fitness_gerais[i]:
            menores_fitness_gerais[i] = menor_fitness_observado
            indice = fitness.index(menor_fitness_observado)
            candidato = populacoes[i][indice]
            print(f"Ilha {i} - Geração {geracoes[i]} - Fitness {menor_fitness_observado} - {''.join(candidato)}")

    if any(geracoes[i] % MIGRACAO_INTERVALO == 0 for i in range(NUM_ILHAS)):
        migracao(populacoes, NUM_MIGRANTES)


Ilha 0 - Geração 1 - Fitness 0 - oxqxo
Ilha 1 - Geração 1 - Fitness 0 - qutuq
Ilha 2 - Geração 1 - Fitness 1 - adfea
Ilha 3 - Geração 1 - Fitness 2 - cuotb
Ilha 4 - Geração 1 - Fitness 2 - otkvo
Ilha 5 - Geração 1 - Fitness 0 - obbbo
Ilha 6 - Geração 1 - Fitness 0 - imbmi
Ilha 7 - Geração 1 - Fitness 1 - pprop
Ilha 8 - Geração 1 - Fitness 0 - ugmgu
Ilha 9 - Geração 1 - Fitness 2 - rcicp


In [13]:
palindromos = []

for i in range(NUM_ILHAS):
    fitness = funcao_objetivo(populacoes[i])
    indice = fitness.index(min(fitness))
    palindromos.append(populacoes[i][indice])

# palindromos agora tem 10 palíndromos otimizados
palindrimos_encontrados = [''.join(palavra) for palavra in palindromos]

print(palindrimos_encontrados)

['oxqxo', 'qutuq', 'adfea', 'cuotb', 'otkvo', 'obbbo', 'imbmi', 'pprop', 'ugmgu', 'rcicp']
